# Clausula JOIN y sus distintos tipos


In [1]:
import sqlite3 
import pandas as pd

conn = sqlite3.connect("db//Chinook_Sqlite.sqlite") 
cursor = conn.cursor()

In [2]:
#Imprimimos el catalogo de tablas 
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall()) 

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [3]:
# Obtenemos las columnas TrackId, el Nombre, y el identificador de album. Imprimiendo los primeros 5 elementros de la consulta
pd.read_sql("SELECT TrackId, Name, AlbumId FROM Track;", conn).head()

,TrackId,Name,AlbumId
0,1,For Those About To Rock (We Salute You),1
1,2,Balls to the Wall,2
2,3,Fast As a Shark,3
3,4,Restless and Wild,3
4,5,Princess of the Dawn,3


In [4]:
# A su vez imprimimos los primeros 5 elementos de la consulta asociada a seleccionar todos los albumes de la tabla Album.
pd.read_sql("SELECT * FROM Album;", conn).head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


Manualmente, nosotros podemos hacer el proceso de ir imprimiendo por pantalla haciendo dos select (por ej: obtener el AlbumId en el ciclo for, y luego hacer una consulta en la tabla Album donde Albumid = 'valor rescatado'). El problema de hacer esto, es que es un proceso ineficiente al hacer multiples consultas consecutivas. Para esto podemos utilizar las clausulas `JOIN` con el fin de hacer este cruce por parte de SQL. Especificamente vamos a ver la primera forma de hacer la union, utilizando la clausula **INNER JOIN**

# INNER JOIN

El `INNER JOIN` va a generar una tabla lógica combinando filas las cuales tengan valores iguales en una o mas tabla. Para hacer el pareo entre filas, necesitamos especificar el campo en común que existe entre las tablas. Con esto, el `INNER JOIN` fuerza a que valores de una tabla A que no se encuentren en una tabla B, sean excluidos. Un diagrama de esta interacción se presenta a continuación:

<center><img src="https://drive.google.com/uc?id=1JQ8x_ojeqgGAuA3gFBhGRIBlx0plXZHI"></center>

La sintaxis para definir un `INNER JOIN` en SQL corresponde a la siguiente:


```SQL
SELECT * FROM Tabla_A INNER JOIN Tabla_B ON Tabla_A.id = Tabla_B.id_a;
SELECT * FROM Tabla_A ta INNER JOIN Tabla_B tb ON ta.id = tb.id_a;
SELECT * FROM Tabla_A ta INNER JOIN Tabla_B tb USING(id); -- Esta clausula funciona siempre y cuando el campo se llama de la misma forma en ambas tablas
```


Realicemos el ejercicio de agregar el nombre del disco para cada canción en la Tabla Track, utilizando la Tabla Albums:

In [5]:
sqlQuery = """
    SELECT t.TrackId, t.Name, a.Title as AlbumTitle, t.UnitPrice 
    FROM Track t 
    INNER JOIN Album a 
    ON t.AlbumId = a.AlbumId;
"""
pd.read_sql(sqlQuery, conn).head()

,TrackId,Name,AlbumTitle,UnitPrice
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.99
1,2,Balls to the Wall,Balls to the Wall,0.99
2,3,Fast As a Shark,Restless and Wild,0.99
3,4,Restless and Wild,Restless and Wild,0.99
4,5,Princess of the Dawn,Restless and Wild,0.99


In [6]:
sqlQuery = """
    SELECT t.TrackId, t.Name, a.Title as AlbumTitle, t.UnitPrice
    FROM Track t 
    INNER JOIN Album a 
    USING(AlbumId);
"""
pd.read_sql(sqlQuery, conn).head()

,TrackId,Name,AlbumTitle,UnitPrice
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.99
1,2,Balls to the Wall,Balls to the Wall,0.99
2,3,Fast As a Shark,Restless and Wild,0.99
3,4,Restless and Wild,Restless and Wild,0.99
4,5,Princess of the Dawn,Restless and Wild,0.99


## Hacer Join con múltiples tablas

SQL otorga flexibilidad para poder hacer `JOIN` hacia múltiples tablas, la idea siempre es hacer un match en la relación existente entre las múltiples tablas. Una sintaxis ejemplo puede ser la siguiente:

```SQL
SELECT * 
FROM Tabla_A ta 
INNER JOIN Tabla_B tb 
ON ta.id = tb.id_a 
INNER JOIN Tabla_C tc
ON tb.id = tc.id_b
```

En este caso, tenemos tres tablas que tienen relaciones logicas entre si, y por ende, dos clausulas `INNER JOIN` y subclausulas `ON`. Veamos dos ejemplos utilizando la base de datos Chinook: Imprimir no solo el nombre del albúm si no que también el Nombre del Género Musical, y el segundo ejemplo añadir el nombre del artista a cada canción.

In [7]:
sqlQuery = """
    SELECT t.TrackId, t.Name, a.Title as AlbumTitle, t.UnitPrice , mt.name as Tipo_medio
    FROM Track t 
    INNER JOIN Album a 
    ON t.AlbumId = a.AlbumId
    INNER JOIN MediaType mt
    on t.MediaTypeId = mt.MediaTypeId
    ;
"""
pd.read_sql(sqlQuery, conn).head()

,TrackId,Name,AlbumTitle,UnitPrice,Tipo_medio
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.99,MPEG audio file
1,2,Balls to the Wall,Balls to the Wall,0.99,Protected AAC audio file
2,3,Fast As a Shark,Restless and Wild,0.99,Protected AAC audio file
3,4,Restless and Wild,Restless and Wild,0.99,Protected AAC audio file
4,5,Princess of the Dawn,Restless and Wild,0.99,Protected AAC audio file


In [8]:
sqlQuery = """
    SELECT at.name as Artista, t.Name, a.Title as AlbumTitle, t.UnitPrice , mt.name as Tipo_medio 
    FROM Track t 
    INNER JOIN Album a 
    ON t.AlbumId = a.AlbumId
    INNER JOIN MediaType mt
    ON t.MediaTypeId = mt.MediaTypeId
    INNER JOIN artist at
    ON a.ArtistId = at.ArtistId
    
    ;
"""
pd.read_sql(sqlQuery, conn).head()

,Artista,Name,AlbumTitle,UnitPrice,Tipo_medio
0,AC/DC,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.99,MPEG audio file
1,Accept,Balls to the Wall,Balls to the Wall,0.99,Protected AAC audio file
2,Accept,Fast As a Shark,Restless and Wild,0.99,Protected AAC audio file
3,Accept,Restless and Wild,Restless and Wild,0.99,Protected AAC audio file
4,Accept,Princess of the Dawn,Restless and Wild,0.99,Protected AAC audio file


Cabe destacar que todos los operadores condicionales, agrupamientos y agregaciones se pueden seguir aplicando:

In [10]:
# Aumentando el Ejemplo 2
sqlQuery = """
    SELECT at.name as Artista, t.Name, a.Title as AlbumTitle, t.UnitPrice , mt.name as Tipo_medio 
    FROM Track t
    
    INNER JOIN Album a 
    ON t.AlbumId = a.AlbumId
    INNER JOIN MediaType mt
    ON t.MediaTypeId = mt.MediaTypeId
    INNER JOIN artist at
    ON a.ArtistId = at.ArtistId
    
    WHERE t.UnitPrice > 1.5    
    ;
"""
pd.read_sql(sqlQuery, conn).head()

,Artista,Name,AlbumTitle,UnitPrice,Tipo_medio
0,Battlestar Galactica,Battlestar Galactica: The Story So Far,Battlestar Galactica: The Story So Far,1.99,Protected MPEG-4 video file
1,Battlestar Galactica,Occupation / Precipice,"Battlestar Galactica, Season 3",1.99,Protected MPEG-4 video file
2,Battlestar Galactica,"Exodus, Pt. 1","Battlestar Galactica, Season 3",1.99,Protected MPEG-4 video file
3,Battlestar Galactica,"Exodus, Pt. 2","Battlestar Galactica, Season 3",1.99,Protected MPEG-4 video file
4,Battlestar Galactica,Collaborators,"Battlestar Galactica, Season 3",1.99,Protected MPEG-4 video file


In [11]:
sqlQuery = """
    SELECT at.name as Artista, COUNT() as cantidad_canciones, AVG(t.Milliseconds)/(60*1000) as duracion_promedio
    FROM Track t

    INNER JOIN Album a 
    ON t.AlbumId = a.AlbumId
    INNER JOIN artist at
    ON a.ArtistId = at.ArtistId
    
    GROUP BY at.name
    HAVING cantidad_canciones > 5
    ORDER BY Artista
    ;
"""
pd.read_sql(sqlQuery, conn).head(10)

,Artista,cantidad_canciones,duracion_promedio
0,AC/DC,18,4.494143
1,Aerosmith,15,4.901899
2,Alanis Morissette,13,4.424263
3,Alice In Chains,12,4.513007
4,Amy Winehouse,23,4.043375
5,Antônio Carlos Jobim,31,3.832465
6,Apocalyptica,8,5.565431
7,Audioslave,40,4.439828
8,BackBeat,12,2.244058
9,Battlestar Galactica,20,46.174409


## Natural Join

Varios motores de base de datos, implementan una forma sencilla de implementar los `INNER JOIN`. En este caso, la clausula `NATURAL JOIN` implementa una busqueda automatica de los atributos a los cuales existe la relación para poder hacer la tabla lógica. En este caso la sintaxis para poder aplicar dicha clausula corresponde a:

```SQL
SELECT * FROM Tabla_A ta NATURAL JOIN Tabla_B tb;
```



Veamoslo con un ejemplo, transformando un `INNER JOIN` a `NATURAL JOIN`:

In [12]:
sqlQuery = """
    SELECT t.TrackId, t.Name, a.Title as AlbumTitle, t.UnitPrice
    FROM Track t 
    NATURAL JOIN Album a;
"""
pd.read_sql(sqlQuery, conn).head()

,TrackId,Name,AlbumTitle,UnitPrice
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.99
1,2,Balls to the Wall,Balls to the Wall,0.99
2,3,Fast As a Shark,Restless and Wild,0.99
3,4,Restless and Wild,Restless and Wild,0.99
4,5,Princess of the Dawn,Restless and Wild,0.99


# Self-Joins

Hasta ahora hemos visto como hacer operaciones `JOIN` entre multiples tablas. Aunque sea poco intuitivo, podemos hacer un join sobre una misma tabla con el fin de explotar relaciones existentes entre campos de una misma tabla. Si hiciesemos un diagrama seria de este tipo: 

<center><img src="https://drive.google.com/uc?id=1c8zq0Acx9-G3IXMTaJulasM_XfcGTfMP"></center>

Para entender que esta pasando aca, veamos de forma practica utilizando la tabla de empleados (Employee):

<center><img src="https://drive.google.com/uc?id=1izk-22a-uUDdqVjvR04X2-efwIZuHTZi"></center>

En este caso, podemos ver que existe una relación donde cada empleado, le reporta a otro empleado (su superior y asi sucesivamente). En este caso, puede hacer sentido la utilización de un **SELF JOIN**. En terminos de sintaxis es similar a los joins vistos anteriormente, solo hay que tener en cuenta los alias de las tablas. Veamoslo utilizando SQLite y Pandas:


In [13]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    INNER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn).head(10)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,Nancy,Edwards,Sales Manager,Andrew Adams,General Manager
1,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
3,Steve,Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael,Mitchell,IT Manager,Andrew Adams,General Manager
5,Robert,King,IT Staff,Michael Mitchell,IT Manager
6,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager


Veamos cuantos elementos retorna la consulta con join y la consulta normal en la tabla de empleados **¿Que esta sucediendo?**

In [15]:
sqlQuery = """
    SELECT COUNT()
    FROM Employee e

    INNER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
print("Consulta con Join ====")
print(pd.read_sql(sqlQuery, conn))


sqlQuery = """
    SELECT COUNT()
    FROM Employee e
    ;
"""
print("Consulta sin Join ====")
print(pd.read_sql(sqlQuery, conn))

Consulta con Join ====
   COUNT()
0        7
Consulta sin Join ====
   COUNT()
0        8


# Left - Right - Outer Joins

En todos los ejemplos que hemos visto hasta ahora, hacen uso del `INNER JOIN`, y tal como vimos en el ejemplo previo a la dinamica, esta clausula tiene algunas desventajas. Si bien puede hacer el match según la relación que nosotros definamos, vamos a dejar a fuera todos los registros que no cumplan la condición perdiendo información en el proceso. Como tal, nosotros no sabiamos quien era `Andrew Adams` (o a quien le reportaba). Para esto existen otras clausulas las cuales provienen de la clausula `OUTER JOIN` y las cuales permiten extraer información de las tablas a las cuales estamos uniendo, sin perder los resultados cuando no existe la relación logica entre tablas. 

## Left Outer Join:

La clausula `LEFT OUTER JOIN`, trae todos los registros que cumplan la condición de la relación entre las tablas **y todos los registros de la tabla que esta a la izquierda del JOIN**. De forma Gráfica tenemos la siguiente imagen:

<center><img src="https://drive.google.com/uc?id=1W32tUlQyNRJsue4wGnLJqJO8ejaKDsN1"></center>

y la sintaxis de dicha clausula es bastante similar a las clausula de `Inner Join`, reemplazando solo la clausula por `Left Outer Join`:

```SQL
SELECT * FROM Tabla_a ta LEFT OUTER JOIN Tabla_b tb on ta.id = tb.id_a
```
En este caso, si para un registro de ta no existe la condicion `ta.id = tb.id_a` de todas formas se retornara el registro de `ta`, pero para las columnas de `tb` retornara valores nulos. Podermos verlo como ejemplo en la consulta de los empleados, donde en vez de el `ÌNNER JOIN` vamos a utilizar un `LEFT OUTER JOIN`

In [16]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn).head(10)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,Andrew,Adams,General Manager,None,None
1,Nancy,Edwards,Sales Manager,Andrew Adams,General Manager
2,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve,Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,Michael,Mitchell,IT Manager,Andrew Adams,General Manager
6,Robert,King,IT Staff,Michael Mitchell,IT Manager
7,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager


y viendo la cantidad de registos que retorna, coincide con la cantidad de elementos que hay en la tabla de empleados.

In [17]:
sqlQuery = """
    SELECT COUNT()
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
print("Consulta con Join ====")
print(pd.read_sql(sqlQuery, conn))

Consulta con Join ====
   COUNT()
0        8


## Right Outer Join

La clausula `RIGHT OUTER JOIN`, trae todos los registros que cumplan la condición de la relación entre las tablas **y todos los registros de la tabla que esta a la Derecha del JOIN**. De forma Gráfica tenemos la siguiente imagen:

<center><img src="https://drive.google.com/uc?id=1Rrr0d1hd858VbsGBKLzLx0kEq42zRkGw"></center>

y la sintaxis de dicha clausula es similar a las clausula de `LEFT OUTER JOIN`, reemplazando solo `LEFT` por `RIGHT` en la clausula:

```SQL
SELECT * FROM Tabla_a ta RIGHT OUTER JOIN Tabla_b tb on ta.id = tb.id_a
```
En este caso, si para un registro de ta no existe la condicion `tb.id_a = ta.id` de todas formas se retornara el registro de `tb`, pero para las columnas de `ta` retornara valores nulos. 

Ahora, existe un **problema en SQLite**... Veamos que sucede si aplicamos un `RIGHT OUTER JOIN` al mismo ejemplo de la tabla de empleados:

In [18]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    RIGHT OUTER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn).head(10)

DatabaseError: Execution failed on sql '
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    RIGHT OUTER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    ;
': RIGHT and FULL OUTER JOINs are not currently supported

La pregunta entonces: **¿Que podemos hacer para tratar de solucionar este problema de implementación?** la respuesta esta en el siguiente codigo:

In [19]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e_reports 

    LEFT OUTER JOIN Employee e
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,Nancy,Edwards,Sales Manager,Andrew Adams,General Manager
1,Michael,Mitchell,IT Manager,Andrew Adams,General Manager
2,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve,Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,None,None,None,Jane Peacock,Sales Support Agent
6,None,None,None,Margaret Park,Sales Support Agent
7,None,None,None,Steve Johnson,Sales Support Agent
8,Robert,King,IT Staff,Michael Mitchell,IT Manager
9,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager


Haciendo ese traspaso, nosotros podemos emular la funcionalidad que existe para la clausula `RIGHT OUTER JOIN`. Solo destacar que esto es un problema que existe en SQLite, tal como detallan en el siguiente documento: [SQL Omitted Features](https://sqlite.org/omitted.html)

## Full Outer Join

La clausula `FULL OUTER JOIN`, corresponde a la union de `LEFT OUTER JOIN` y de `RIGHT OUTER JOIN`. Esto quiere decir que va a incluir los registros de ambas tablas al momento de hacer el proceso de emparejamiento segun las relaciones lógicas

<center><img src="https://drive.google.com/uc?id=18g1khXUtjc2wMwjxmyarypoKICOzrACz"></center>

La sintaxis para esta clausula es igual para `LEFT OUTER JOIN` y `RIGHT OUTER JOIN`:

```SQL
SELECT * FROM Tabla_a ta FULL OUTER JOIN Tabla_b tb on ta.id = tb.id_a
```

El problema nuevamente, ocurre con SQLite no implementando `FULL OUTER JOIN` por lo que tendremos que hacer una solución alterna tal como se hizo para el `RIGHT OUTER JOIN`. Adelantandonos un poco en la materia, tendremos que utilizar un operador que es el operador `UNION` el cual combina multiples resultados de varias consultas `SELECT`. En este caso para poder emular el `FULL OUTER JOIN`, tenemos que hacer lo siguiente:

In [20]:
sqlQuery1 = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    
"""

sqlQuery2 = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e_reports 

    LEFT OUTER JOIN Employee e
    ON e.ReportsTo = e_reports.EmployeeId
    
"""

sqlQuery = sqlQuery1 + '    UNION\n' + sqlQuery2
print(sqlQuery)


    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports 
    ON e.ReportsTo = e_reports.EmployeeId
    
    UNION

    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e_reports 

    LEFT OUTER JOIN Employee e
    ON e.ReportsTo = e_reports.EmployeeId
    



In [21]:
pd.read_sql(sqlQuery, conn)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,None,None,None,Jane Peacock,Sales Support Agent
1,None,None,None,Laura Callahan,IT Staff
2,None,None,None,Margaret Park,Sales Support Agent
3,None,None,None,Robert King,IT Staff
4,None,None,None,Steve Johnson,Sales Support Agent
5,Andrew,Adams,General Manager,None,None
6,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
7,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager
8,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
9,Michael,Mitchell,IT Manager,Andrew Adams,General Manager
